In [ ]:
#To see the current directory 
#chdir would change the directory
import os

os.chdir('../input/datafolder')
os.listdir()

In [ ]:
#Data Class for custom data implemnetation in pytorch
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import os
import skimage as io
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt


class MyDataset(Dataset):
    def __init__(self,root_dir,file_path,transform = None):
        self.root_dir = root_dir
        
        #Reading CSV File
        self.data_files = pd.read_csv(file_path, header = None) 
        
        self.file_path = file_path
        self.transform = transform

    def __getitem__(self, idx): 
        if torch.is_tensor(idx):
            idx = idx.tolist()
        #Contains list of path of all images    
        img_path = os.path.join(self.root_dir,self.data_files.iloc[idx,0])  
        
        #Read all the images and convert them in RGB
        image = Image.open(img_path).convert("RGB") 
        
        #Label which is the 2nd column
        label = int(self.data_files.iloc[idx,1]) 
        
        if self.transform:
            img = self.transform(image)
            
        #Return a list of miage and its label
        return img, label

    def __len__(self):
        return len(self.data_files)

In [ ]:
#Deinifing tranforms to input image (Sometimes called Image Augmentation)
#This improves the robustness of the classification by cropping, flipping etc.
trans = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.RandomRotation(degrees = (-180,180), resample=False, expand=False, center=None, fill=None),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
            transforms.ToTensor()
                                     ])

#My Datasets for training
img_dir = 'train_renamed'
csv_path = 'train_renamed.csv'
dataset = MyDataset(root_dir = img_dir, file_path = csv_path, transform = trans )


#Dataset for Valdiation
val_dir = 'val'
val_path = 'validation.csv'
testset = MyDataset(root_dir = val_dir, file_path = val_path, transform = trans )

In [ ]:
#DataLoader
trainloader = DataLoader(dataset,batch_size = 16, num_workers=4, shuffle = True,pin_memory=True)
testloader = DataLoader(testset,batch_size = 4, num_workers=4, shuffle = True,pin_memory=True)

In [ ]:
#Iterating through the train Dataloader
batch = iter(trainloader).next()

data = batch[0]
labels = batch[1]

In [ ]:
#Iterating through the test Dataloader
testbatch = iter(testloader).next()

images = testbatch[0]
val_labels = testbatch[1]

In [ ]:
#Checking image type and shape 
print(type(batch))
print(data.shape)
print(labels)

In [ ]:
#Image Show

for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(data[i, 0], cmap='gray', vmin=0., vmax=1.)
    plt.title('Label: {}'.format(labels[i]))

In [ ]:


from model import Network

            

In [ ]:
#Network

#from model import Network
import torch.optim as optim
import torch.nn as nn

#Creating Instance of the Network
net = Network()

#Choosing Omptimizer 
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, nesterov = True)
#optimizer = optim.ASGD(net.parameters(), lr=0.01, lambd=0.0001, alpha=0.5, t0=1000000.0)
#optimizer = optim.Adadelta(net.parameters(), lr=0.05, rho=0.9, eps=1e-06)
#optimizer = optim.Adam(net.parameters(), lr=0.0001)

#Choosing Loss for Classifcation
criterion = nn.CrossEntropyLoss()


#To utlise gpu
if torch.cuda.is_available():
    net = net.cuda()
    criterion = criterion.cuda()

print(net)

In [ ]:
#Training The Model
for i in range(400):
    t_running_loss = 0
    v_running_loss = 0
    for data, labels in trainloader:

        if torch.cuda.is_available():
            data = data.cuda()
            labels = labels.cuda()

        # Training pass
        optimizer.zero_grad()
        
        output = net(data)
        
        t_loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        t_loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        t_running_loss += t_loss.item()
        
    for data, labels in testloader:

        if torch.cuda.is_available():
            data = data.cuda()
            labels = labels.cuda()
        
        output = net(data)
        v_loss = criterion(output, labels)
        
        v_running_loss += v_loss.item()

    else:
        
        if (v_running_loss/len(testloader))< 0.3:
            break
        else:
            print("Epoch {} - Training loss: {}".format(i+1, t_running_loss/len(trainloader)))
            print("Epoch {} - Validation loss: {}".format(i+1, v_running_loss/len(testloader)))

In [ ]:
import numpy as np

#To load saved model uncomment
#net.load_state_dict(torch.load('model'))

net.eval()
prediction = []

# getting predictions on test set and measuring the performance
correct_count, all_count = 0, 0
for data,labels in testloader:
    for i in range(len(labels)):
        if torch.cuda.is_available():
            data = data.cuda()
            labels = labels.cuda()
        img = data[i].view(1, 3, 128, 128)
        with torch.no_grad():
            logps = net(img)


        ps = torch.exp(logps)
        probab = list(logps.cpu()[0])
        pred_label = probab.index(max(probab))
        prediction.append(pred_label)
        true_label = labels.cpu()[i]
        if(true_label == pred_label):
          correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

In [ ]:
#To save the predications as panda data Frame and exporting csv

import glob
import re
from PIL import Image
from torch.autograd import Variable
from torchvision import datasets, transforms
#from model import Network
import pandas as pd

#os.chdir("../input/mydata")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loader = transforms.Compose([transforms.Resize((128,128)),transforms.ToTensor()])
#net = model.Network().to(device)
#net.load_state_dict(torch.load('model'))
#net = model_conv

net.eval()
filelist = glob.glob('val/*.png')
filelist.sort()

predten = torch.tensor([],dtype = torch.float).to(device)
i = 0
label = []
df = pd.DataFrame(columns = ['Id','Category'])  

for fname in filelist:
    image = Image.open(fname).convert('RGB')
    image = loader(image).float()

    image = Variable(image)
    image = image.unsqueeze(0)

    image = image.to(device)
    pred = net(image)
    
    predicted_label = pred.argmax(dim = 1)

    predten = torch.cat((predten,predicted_label),dim = 0)
    
    
    df1 = pd.DataFrame({'Id': fname , 'Category' : predicted_label.cpu() } )
    df = df.append(df1)



    i += 1
    
print("Number of Images Validated: ",i)

In [ ]:

#save model
os.chdir("/kaggle/working/") 
net.save_model()



In [ ]:
df.to_csv('Kaggle_99375.csv', index = False , header = True)